## Prepare invironment

In [1]:
import os
import datetime
import fnmatch
import re
import astromatic_wrapper as aw #TODO: install aw to itaf virtual env
from astropy.io import fits

In [2]:
#!pip install apex

In [3]:
import pyraf.iraf as iraf
import pyraf

In [4]:
import sys
sys.path.insert(0, '/home/ser/Dev/astro_engine/code')
from astro_utils import AEDirsTreeConfigurer

In [7]:
from fits_utils import *

In [5]:
print_iraf_tasks = 0
iraf.module.images(_doprint=print_iraf_tasks) # print or not to print (why it not working?)

Just test that PyRAF works

In [3]:
print iraf.module.imaccess('im_3.fit') #TODO: implement access check if needed
input_fits = 'im_1.fit, im_2.fit'
output_fits = 'res.fit'
s = iraf.module.imcombine(input_fits, output_fits, combine='median', Stdout=1, Stderr=1)

0


In [75]:
dirs = AEDirsTreeConfigurer()#images_path='images/GRB130427A')
dirs.build_dirs_tree()
dirs.new_log_dir()
paths = dirs.get_paths()
paths

{'catalogs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/catalogs',
 'config': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config',
 'images': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images',
 'logs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/logs',
 'stacks': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/stacks',
 'temp': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp'}

In [108]:
# set image filename
files = {'image': os.path.join(paths['images'], 'GRB130427_R60_001_003.fit')}

In [36]:
# correct flats headers
#from functools import partial
#p = '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp/flat'
#f = filter_fits_by_header(input_images_names, paths['temp']+'/flat', EXPTYPE='Light')
#change_header_apply = partial(change_header, fits_path=p, EXPTYPE=to_header_str_format('Flat'))
#apply_to_fits(f, p, change_header_apply)

In [152]:
# take lists of each type images
input_images_names = sorted(get_fits_images_from_dir(os.path.join(paths['temp']), 'fit', True, True))
bias_list = filter_fits_by_header(input_images_names, paths['temp'], EXPTYPE='Bias')
dark_list = filter_fits_by_header(input_images_names, paths['temp'], EXPTYPE='Dark')
flat_list = filter_fits_by_header(input_images_names, paths['temp'], EXPTYPE='Flat')
images_list = filter_fits_by_header(input_images_names, paths['temp'], EXPTYPE='Light')

In [153]:
# superbias
iraf.noao.imred(_doprint=0)
iraf.noao.imred.ccdred(_doprint=0)
iraf.noao.imred.ccdred.instrument = os.path.join(paths['config'], 'instrument.dat')
iraf.noao.imred.ccdred.zerocombine.setParam('ccdtype', "")  
gain = fits.getheader(bias_list[0], 0)['GAIN']
iraf.noao.imred.ccdred.zerocombine.setParam('gain', gain) 
bias_list_txt = os.path.join(paths['temp'], 'super', 'bias_list.txt')
list_to_file(bias_list, bias_list_txt)
superbias_filename = os.path.join(paths['temp'], 'super', 'superbias.fits')
iraf.noao.imred.ccdred.zerocombine(input='@' + bias_list_txt, output=superbias_filename, 
                                       process='no', delete='no', clobber='no')

In [155]:
# compute darks_exp_times, key:filename, value:exp_time
darks_exp_times_dict = header_keywords_values(dark_list, keywords=['EXPTIME'])
if 'EXPTIME' in darks_exp_times_dict.keys():
    darks_exp_times = sorted(darks_exp_times_dict['EXPTIME'].keys())
else:
    darks_exp_times = []
print 'darks exp times:', darks_exp_times

# for each image filter, find best dark for flat and flat for image
filters = ['R']
filters_exp_times_mappings = {}
for filter_ in filters:
    print 'filter:', filter_
    # filter fits with current filter
    filter_flat_list = filter_fits_by_header(flat_list, paths['temp'], FILTER=filter_)
    filter_images_list = filter_fits_by_header(images_list, paths['temp'], FILTER=filter_)
    
    # compute flats_exp_times
    flats_exp_times_dict = header_keywords_values(filter_flat_list, keywords=['EXPTIME'])
    if 'EXPTIME' in flats_exp_times_dict.keys():
        flats_exp_times = sorted(flats_exp_times_dict['EXPTIME'].keys())
    else:
        flats_exp_times = []
    print 'flats exp times:', flats_exp_times

    # compute images_exp_times
    images_exp_times_dict = header_keywords_values(filter_images_list, keywords=['EXPTIME'])
    if 'EXPTIME' in images_exp_times_dict.keys():
        images_exp_times = sorted(images_exp_times_dict['EXPTIME'].keys())
    else:
        images_exp_times = []
    print 'images exp times:', images_exp_times

    # search for best dark_exp_times for images
    images_darks_exp_times = best_exp_time(images_exp_times, darks_exp_times)
    print "best darks exp time for images:", images_darks_exp_times

    # search for best dark_exp_times for flats
    flat_darks_exp_times = best_exp_time(flats_exp_times, darks_exp_times)
    print "best darks exp time for flats:", flat_darks_exp_times

    # search for best flat_exp_times for images
    images_flats_exp_times = best_exp_time(images_exp_times, flats_exp_times)
    print "best flats exp time for images:", images_flats_exp_times
    
    filters_exp_times_mappings[filter_] = {'images-dark': images_darks_exp_times, 
                                           'flat-dark': flat_darks_exp_times,
                                           'images-flat': images_flats_exp_times}

darks exp times: [2.0, 60.0]
filter: R
flats exp times: []
images exp times: [60.0]
best darks exp time for images: {60.0: 60.0}
best darks exp time for flats: {}
best flats exp time for images: {}


In [156]:
filters_exp_times_mappings

{'R': {'flat-dark': {}, 'images-dark': {60.0: 60.0}, 'images-flat': {}}}

In [178]:
# when we know which dark needed for which flat and which image, so compute all needed darks
exp_times_superdarks = {}
for filter_, mapps in filters_exp_times_mappings.iteritems():
    for mapp, vals in mapps.iteritems():
        for_, by_ = mapp.split('-')[0], mapp.split('-')[1]
        if vals and by_ == 'dark':
            for k, v in vals.iteritems():
                superdark_filename = os.path.join(paths['temp'], 'super',
                                                  'superdark_%s_%s_%d.fits' % (filter_, for_, k))
                input_darks_list = darks_exp_times_dict['EXPTIME'][v]
                scale_coeff = float(k) / float(v)
                iraf.noao.imred(_doprint=0)
                iraf.noao.imred.ccdred(_doprint=0)
                iraf.noao.imred.ccdred.instrument = os.path.join(paths['config'], 'instrument.dat')
                iraf.noao.imred.ccdred.darkcombine.setParam('ccdtype', "")  
                gain = fits.getheader(input_darks_list[0], 0)['GAIN']
                iraf.noao.imred.ccdred.darkcombine.setParam('gain', gain)
                iraf.noao.imred.ccdred.darkcombine.setParam('scale', 'exposure')
                iraf.noao.imred.ccdred.darkcombine.setParam('combine', 'median')
                iraf.noao.imred.ccdred.darkcombine.setParam('process', 'no')
                dark_list_txt = os.path.join(paths['temp'], 'super', 'dark_list.txt')
                list_to_file(input_darks_list, dark_list_txt)
                iraf.noao.imred.ccdred.darkcombine(input='@' + dark_list_txt, output=superdark_filename)
                # TODO: subtract superbias
                
                #iraf.module.imarith.lParam()
                sub_1 = 'temp/super/superdark.txt'
                sub_2 = 'temp/super/superbias.txt'
                list_to_file([superdark_filename], sub_1)
                list_to_file([superbias_filename], sub_2)
                iraf.module.imarith('@'+sub_1, '-', '@'+sub_2, 'res.fits')
                
# 
for filter_, mapps in filters_exp_times_mappings.iteritems():
    print 'filter:', filter_
    for mapp, vals in mapps.iteritems():
        for_, by_ = mapp.split('-')[0], mapp.split('-')[1]
        if not vals:
            print 'don\'t use', by_, 'for', for_, 'for this filter'
        else:
            for k, v in vals.iteritems():
                coef_str = 'coeff = %f' %(float(k)/float(v)) if by_ == 'dark' else ''
                print 'use', by_, 'with exp_time', v, 'for', for_, 'with exp_time', k, coef_str

/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp/super/superdark_R_images_60.fits is not an image or a number


filter: R
use dark with exp_time 60.0 for images with exp_time 60.0 coeff = 1.000000
don't use flat for images for this filter
don't use dark for flat for this filter


In [102]:
# compute superdark for each flat exp_times
for exp_time in 
    

{2.0: 12, 60.0: 60}

In [101]:
def best_exp_time(exp_list_for, exp_list_by):
    res = {}
    if not exp_list_by or not exp_list_for:
        return res
    for iet in exp_list_for:
        min_diff = abs(exp_list_by[0] - iet)
        best_det = exp_list_by[0]
        for det in exp_list_by:
            diff = abs(det - iet)
            if diff < min_diff:
                best_det = det
                min_diff = diff
        res[iet] = best_det
    return res

In [ ]:
# superdarks
exp_times = ['2.', '60.']

<h2>Pipeline (local version)</h2>

<h3>Import all needed packages</h3>

In [1]:
import os
import datetime
import copy
import astromatic_wrapper as aw #TODO: install aw to itaf virtual env
from astropy.io import fits
import pyraf
from pyraf import iraf
import sys
sys.path.insert(0, '/home/ser/Dev/astro_engine/code')
from astro_utils import AEDirsTreeConfigurer, AEJsonConfigLoader
from fits_utils import *

<h3>Define pipeline parameters</h3>

Define dirs tree

In [2]:
dirs = AEDirsTreeConfigurer()#images_path='images/GRB130427A')
dirs.build_dirs_tree()
dirs.new_log_dir()
paths = dirs.get_paths()
paths

{'catalogs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/catalogs',
 'config': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/config',
 'images': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images',
 'logs': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/logs',
 'stacks': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/stacks',
 'temp': '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp'}

Define config for astro software from json config files

In [171]:
all_configs = AEJsonConfigLoader(os.path.join(paths['config'], 'common_config.json'))
print all_configs.get_required_common_params()
all_configs.build_config()
pipeline_config = all_configs.get_pipeline_config()

{'config_files': ['pipeline_config']}


In [4]:
for key in pipeline_config['pipeline_stages'].keys():
    print "-----",key
    for kkey in pipeline_config['pipeline_stages'][key]['stage_steps'].keys():
        print kkey
    print '-----------'

----- preparation
fits_preprocessing_common
tech_frames_preprocessing
-----------
----- images_calibration
create_calibration_headers
basic_reduction
create_calibrated_images_catalogs
calibrate_images
model_psf_calibrated_images_catalogs
create_calibration_catalogs
create_calibrated_images_catalogs_psf
-----------
----- images_coaddition
model_psf_coadded_image_catalog
coadd_calibrated_images
create_coadded_image_catalog
create_coadded_image_catalog_psf
-----------


In [5]:
#EXPTYPE - FITS header keywords for type of frame
#Bias - bias
#Dark - dark
#Light - flat

In [6]:
print(pyraf.__version__)
#iraf.pwd()
#from pyraf import irafinst
#irafinst.getIrafVer()

#iraf.files(bias_list)

#iraf.images(_doprint=0) # load images
#iraf.imutil(_doprint=0) # load imutil
#iraf.imheader.setParam('longheader', 'yes')
#iraf.imheader(bias_list[0])

2.2.dev


In [7]:
def list_to_file(list_, filename, sep='\n'):
    with open(filename, 'w') as fid:
        for item in list_:
            fid.write('%s%s' % (str(item), sep))

def to_list(something):
    return something if isinstance(something, list) else [something]

def filter_fits_by_header(fits_filenames, fits_path=None, match_any=False, ignore_case=True,
                          ignore_edge_spaces=True, hdu_num=0, **kwargs):
    iffs = [os.path.join(fits_path, os.path.basename(iff)) if fits_path is not None else iff\
            for iff in to_list(fits_filenames)]
    res = []
    for iff in iffs:
        fits_file_header = fits.getheader(iff, hdu_num)
        keywords_match = 0
        for keyword, value in kwargs.iteritems():
            if keyword in fits_file_header:
                header_val = str(fits_file_header[keyword]).strip() if ignore_edge_spaces \
                else str(fits_file_header[keyword])
                header_val = header_val.lower() if ignore_case else header_val
                kwargs_val = str(value).strip() if ignore_edge_spaces else str(value)
                kwargs_val = kwargs_val.lower() if ignore_case else kwargs_val
                if header_val == kwargs_val:
                    keywords_match += 1
                    if match_any:
                        break
        if (keywords_match and match_any) or (keywords_match == len(kwargs)):
            res.append(iff)
    return res

def header_keywords_values(fits_filenames, fits_path=None, keywords=['SIMPLE'], keys_keywords=True, lower_case=True,
                           split_edge_spaces=True, hdu_num=0):
    iffs = [os.path.join(fits_path, os.path.basename(iff)) if fits_path is not None else iff \
            for iff in to_list(fits_filenames)]
    kws = to_list(keywords)
    res = {}
    for iff in iffs:
        fits_file_header = fits.getheader(iff, hdu_num)
        if keys_keywords:
            for keyword in kws:
                if keyword not in res.keys():
                    res[keyword] = {}
                if keyword in fits_file_header:
                    val = fits_file_header[keyword]
                    if val not in res[keyword].keys():
                        res[keyword][val] = []
                    res[keyword][val].append(iff)
        else:   
            for keyword in kws:
                if keyword in fits_file_header:
                    res[iff] = {keyword: fits_file_header[keyword]}
                else:
                    res[iff] = {keyword: None}
    return res

def change_header(fits_filename, fits_path=None, new_path=None, clobber=True, **kwargs):
    iff = os.path.join(fits_path, os.path.basename(fits_filename)) if fits_path is not None else fits_filename
    data, header = fits.getdata(iff, header=True)
    for keyword, value in kwargs.iteritems():
        header[keyword] = value
    if new_path is None:
        new_path = fits_path
    iff = os.path.join(new_path, os.path.basename(iff))
    fits.writeto(iff, data, header, clobber=clobber)
    return iff

def to_header_str_format(str_value, length=8):
    return str_value.strip().ljust(length)

def apply_to_fits(fits_filenames, fits_path=None, func=None):
    iffs = [os.path.join(fits_path, os.path.basename(iff)) if fits_path is not None else iff \
            for iff in to_list(fits_filenames)]
    if not func:
        return iffs
    res = []
    for iff in iffs:
        res.append(func(iff))
    return res

def images_stat(images, fields='mode,mean', return_dict=True, stdout=1, stderr=2):
    iraf.images(_doprint=0)
    if return_dict:
        iraf.images.imstat.setParam('format', 'no')
        #iraf.images.imstat.setParam('')
    iraf.images.imstat.setParam('fields', fields)
    images = images if isinstance(images, list) else [images]
    res = []
    for image in images:
        res.append(iraf.imstat(image, Stdout=stdout, Stderr=stderr))
    return res

In [8]:
#from functools import partial
#p = '/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/temp'
#f = filter_fits_by_header(input_images_names, paths['temp'], EXPTYPE='DARK')
#change_header_apply = partial(change_header, fits_path=p, EXPTYPE=to_header_str_format('Bias'))
#apply_to_fits(f, p, change_header_apply)

<h4>Basic reduction</h4>

In [9]:
def compute_superbias(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    software_params = step_params_config['software_params']
    
    input_tech_frames_dir = paths[step_params_config['input_params']['tech_frames']['frames_dir']]
    output_tech_frames_dir = paths[step_params_config['output_params']['tech_frames']['frames_dir']]

    input_bias_config = step_params_config['input_params']['tech_frames']['bias']
    input_bias_config['format'] = to_ext_format(input_bias_config['format'])
    output_bias_config = step_params_config['output_params']['tech_frames']['bias']
    output_bias_config['format'] = to_ext_format(output_bias_config['format'])
    
    input_bias_dir = os.path.join(input_tech_frames_dir, input_bias_config['sub_dir'])
    output_bias_dir = os.path.join(output_tech_frames_dir, output_bias_config['sub_dir'])

    superbias_filename = os.path.join(output_bias_dir,
                                      (output_bias_config['name'] + output_bias_config['format']))

    filter_bias_kwargs = {input_bias_config['type_keyword']: input_bias_config['type_keyword_value']}
    input_bias_filenames = filter_fits_by_header(input_images_names, input_bias_dir, 
                                                 **filter_bias_kwargs)
    
    iraf.noao.imred(_doprint=0)
    iraf.noao.imred.ccdred(_doprint=0)
    iraf.noao.imred.ccdred.instrument = os.path.join(paths[software_params['config']['INSTRUMENT_DIR']],
                                                     software_params['config']['INSTRUMENT_FILE'])
    
    if 'ccdtype' in input_bias_config.keys():
        iraf.noao.imred.ccdred.zerocombine.setParam('ccdtype', input_bias_config['ccdtype'])
    else:
        iraf.noao.imred.ccdred.zerocombine.setParam('ccdtype', '')
    if 'gain_keyword' in input_bias_config.keys():
        gain = fits.getheader(input_bias_filenames[0], 0)[input_bias_config['gain_keyword']]
        iraf.noao.imred.ccdred.zerocombine.setParam('gain', gain)
    
    bias_list = os.path.join(paths['temp'], 'bias_list.txt')
    list_to_file(input_bias_filenames, bias_list)
    
    iraf.noao.imred.ccdred.zerocombine(input='@' + bias_list, output=superbias_filename, 
                                       process='no', delete='no', clobber='no')
    
    return superbias_filename

In [10]:
def compute_supers(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    input_params_config = step_params_config['input_params']
    output_params_config = step_params_config['output_params']
    
    # prepare io params 
    exp_times = to_list(input_params_config['images_exp_times_secs'])
    filters = to_list(input_params_config['images_filters'])
    images_filter_keyword = input_params_config['images_filter_keyword']
    input_tech_frames_dir = paths[input_params_config['tech_frames']['frames_dir']]
    output_tech_frames_dir = paths[output_params_config['tech_frames']['frames_dir']]
    use_bias = 'bias' in input_params_config['tech_frames']['use_frames']
    use_dark = 'dark' in input_params_config['tech_frames']['use_frames']
    use_flat = 'flat' in input_params_config['tech_frames']['use_frames']

    
    if use_bias:
        superbias_filename = compute_superbias(input_images_names, paths, 
                                               pipeline_config, pipeline_stage, pipeline_step)   
    if use_dark:
        # compute superdarks for all exp_times for data images
        #if no exp_time, take exp_time with biggest list of corresponding darks
        exp_times_darks = {}
        for exp_time in exp_times:
            exp_times_darks[exp_time] = compute_superdark(input_images_names, paths, 
                                               pipeline_config, pipeline_stage, pipeline_step)


In [11]:
#input_images_names = sorted(get_fits_images_from_dir(os.path.join(paths['temp']), 'fit', True, True))
#compute_supers(input_images_names, paths, pipeline_config, 'preparation', 'tech_frames_preprocessing')

<h4>Catalog extraction basic function</h4>

In [12]:
def create_catalogs(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    
    # use paths to fill path-depended params TODO: preprocess config to do this
    step_params_config['software_params']['config_file'] = os.path.join(paths['config'],
                                                                step_params_config['software_params']['config_file'])
    step_params_config['software_params']['temp_path'] = paths[step_params_config['software_params']['temp_path']]
    step_params_config['software_params']['config']['FILTER_NAME'] = os.path.join(paths['config'],
                                                      step_params_config['software_params']['config']['FILTER_NAME'])
    
    # prepare io params   
    # input images
    input_images_format = to_ext_format(step_params_config['input_params']['images_format'])
    input_images_dir = paths[step_params_config['input_params']['images_dir']]
    iffs = build_filenames_list(input_images_names, new_ext=input_images_format, 
                                new_path=input_images_dir)
    
    # input psf models, if exist
    look_for_psf = False
    ipfs = range(len(iffs))
    if 'psf_models_format' in step_params_config['input_params'].keys() and \
    'psf_models_dir' in step_params_config['input_params'].keys():
        look_for_psf = True
        input_psf_models_format = to_ext_format(step_params_config['input_params']['psf_models_format'])
        input_psf_models_dir = paths[step_params_config['input_params']['psf_models_dir']]
        ipfs = build_filenames_list(input_images_names, new_ext=input_psf_models_format, 
                                    new_path=input_psf_models_dir)
    
    # output catalogs
    output_catalogs_format = to_ext_format(step_params_config['output_params']['catalogs_format'])
    output_catalogs_dir = paths[step_params_config['output_params']['catalogs_dir']]
    ocfs = build_filenames_list(input_images_names, new_ext=output_catalogs_format, 
                                    new_path=output_catalogs_dir)
    
    
    if 'check_params_for_scamp_required' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['check_params_for_scamp_required'] == 'true':
            missing_req_scamp_params = list(set(pipeline_config['scamp_required_catalog_params']) - \
            set(step_params_config['software_params']['params']))
            step_params_config['software_params']['params'].extend(missing_req_scamp_params)
    
    if 'check_params_for_psfex_required' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['check_params_for_psfex_required'] == 'true':
            missing_req_psfex_params = list(set(pipeline_config['psfex_required_catalog_params']) - \
            set(step_params_config['software_params']['params']))
            step_params_config['software_params']['params'].extend(missing_req_psfex_params)
    
    # output is a list of catalogs full pathnames
    catalogs = []
    
    for (iff, ipf, ocf) in zip(iffs, ipfs, ocfs):
        software_params = step_params_config['software_params'] # TODO: deepcopy or not?
        # TODO: is it needed or SExtractor recognizes it by itself?
        if 'params_from_fits_header' in step_params_config['data_handle'].keys():
            fits_file = fits.open(iff)
            fits_file_header = fits_file[0].header
            for param_name, fits_header_keyword in step_params_config['data_handle']['params_from_fits_header'].iteritems():
                if fits_header_keyword in fits_file_header:
                    software_params['config'][param_name] = str(fits_file[0].header[fits_header_keyword])
        software_params['config']['CATALOG_NAME'] = ocf
        if look_for_psf:
            software_params['config']['PSF_NAME'] = ipf
        sextractor = aw.api.Astromatic(**software_params)
        #this_cmd, kwargs2 = sextractor.build_cmd(iff, **software_params)
        #print this_cmd
        sextractor.run(iff) #TODO: handle run status (maybe return it?)
        catalogs.append(ocf)
    
    return catalogs

<h4>Calculate calibration basic function</h4>

In [13]:
def calculate_calibration(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    
    # use paths to fill path-depended params TODO: preprocess config to do this
    step_params_config['software_params']['config_file'] = os.path.join(paths['config'],
                                                                step_params_config['software_params']['config_file'])
    step_params_config['software_params']['temp_path'] = paths[step_params_config['software_params']['temp_path']]
    
    # prepare io params   
    # input catalogs
    input_catalogs_format = to_ext_format(step_params_config['input_params']['catalogs_format'])
    input_catalogs_dir = paths[step_params_config['input_params']['catalogs_dir']]
    icfs = build_filenames_list(input_images_names, new_ext=input_catalogs_format, 
                                    new_path=input_catalogs_dir)
    
    # output is empty list
    result = []
    
    calculate_calibration_separately = False
    if 'calculate_calibration_separately' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['calculate_calibration_separately'] == 'true':
            calculate_calibration_separately = True
            
    software_params = step_params_config['software_params'] # TODO: deepcopy or not?
    
    if calculate_calibration_separately:
        for icf in icfs:
            scamp = aw.api.Astromatic(**software_params)
            #this_cmd, kwargs2 = scamp.build_cmd(icf, **software_params)
            #print this_cmd
            scamp.run(icf)
    else:
        scamp = aw.api.Astromatic(**software_params)
        scamp.run(' '.join(icfs)) # TODO: create file with catalogs list, and pass as @catalogs_list
        
    return result

<h4>Image SWarp basic function</h4>

In [14]:
from operator import itemgetter
from itertools import *
def swarp_images(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    
    # use paths to fill path-depended params TODO: preprocess config to do this
    step_params_config['software_params']['config_file'] = os.path.join(paths['config'],
                                                                step_params_config['software_params']['config_file'])
    step_params_config['software_params']['temp_path'] = paths[step_params_config['software_params']['temp_path']]
    step_params_config['software_params']['config']['RESAMPLE_DIR'] = paths[step_params_config['software_params']['config']['RESAMPLE_DIR']]
    
    # prepare io params
    # input images
    input_images_format = to_ext_format(step_params_config['input_params']['images_format'])
    input_images_dir = paths[step_params_config['input_params']['images_dir']]
    iffs = build_filenames_list(input_images_names, new_ext=input_images_format, 
                                new_path=input_images_dir)
    
    # output images
    output_images_format = to_ext_format(step_params_config['output_params']['images_format'])
    output_images_dir = paths[step_params_config['output_params']['images_dir']]
    
    # output weightmaps
    output_weightmaps_format = to_ext_format(step_params_config['output_params']['weightmaps_format'])
    output_weightmaps_dir = paths[step_params_config['output_params']['weightmaps_dir']]
    
    # output is a tuple of lists: swarped images full pathnames and weightmaps full pathnames
    swarped_images, weightmaps = [], []
    
    swarp_separately = False
    if 'swarp_separately' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['swarp_separately'] == 'true':
            swarp_separately = True
            
    software_params = step_params_config['software_params'] # TODO: deepcopy or not?
                   
    if swarp_separately:
        # prepare output_images_filenames and output_weightmaps_filenames (convert them to full pathname)
        offs = build_filenames_list(input_images_names, new_ext=output_images_format, 
                                    new_path=output_images_dir)
        owmfs = build_filenames_list(input_images_names, new_ext=output_weightmaps_format, 
                                    new_path=output_weightmaps_dir)
        
        for (iff, off, owmf) in zip(iffs, offs, owmfs):
            software_params['config']['IMAGEOUT_NAME'] = off
            software_params['config']['WEIGHTOUT_NAME'] = owmf
            swarp = aw.api.Astromatic(**software_params)
            swarp.run(iff)
            swarped_images.append(off)
            weightmaps.append(owmf)   
    else:
        # build coadded image and weightmap names
        output_filename = pipeline_config['observation_id']
        coadd_images_nums = sorted(
            [os.path.basename(iff).rstrip(input_images_format).rsplit('_',1)[1] for iff in iffs])
        nums_ranges = []
        for k, g in groupby(enumerate(coadd_images_nums), lambda (i,x):int(i)-int(x)):
            nums_range = map(itemgetter(1),g)
            nums_ranges.append(nums_range[0] if len(nums_range) == 1 else nums_range[0] + '-' + nums_range[-1])
        nums_ranges = '_'.join(nums_ranges)
        output_image_name = output_filename + '_' + nums_ranges + output_images_format
        output_image_name = os.path.join(output_images_dir, output_image_name)
        output_weightmap_name = output_filename + '_' + nums_ranges + output_weightmaps_format
        output_weightmap_name = os.path.join(output_weightmaps_dir, output_weightmap_name)
        # run SWarp
        software_params['config']['IMAGEOUT_NAME'] = output_image_name
        software_params['config']['WEIGHTOUT_NAME'] = output_weightmap_name
        swarp = aw.api.Astromatic(**software_params)
        swarp.run(' '.join(iffs)) # TODO: create file with images list, and pass as @images_list
        swarped_images.append(output_image_name)
        weightmaps.append(output_weightmap_name)
    
    return swarped_images, weightmaps

<h4>Calculate PSF basic function</h4>

In [15]:
def calculate_psf(input_images_names, paths, pipeline_config, pipeline_stage, pipeline_step):
    # TODO: check config, stage and step for needed for this function keys
    # select pipeline stage and step in config
    step_config = copy.deepcopy(pipeline_config['pipeline_stages'][pipeline_stage]['stage_steps'][pipeline_step])
    step_params_config = step_config['step_params']
    
    # use paths to fill path-depended params TODO: preprocess config to do this
    step_params_config['software_params']['config_file'] = os.path.join(paths['config'],
                                                                step_params_config['software_params']['config_file'])
    step_params_config['software_params']['temp_path'] = paths[step_params_config['software_params']['temp_path']]
    
    # prepare io params   
    # input catalogs
    input_catalogs_format = to_ext_format(step_params_config['input_params']['catalogs_format'])
    input_catalogs_dir = paths[step_params_config['input_params']['catalogs_dir']]
    icfs = build_filenames_list(input_images_names, new_ext=input_catalogs_format, 
                                    new_path=input_catalogs_dir)
    
    # output psf models
    output_psf_models_format = to_ext_format(step_params_config['output_params']['psf_models_format'])
    output_psf_models_dir = paths[step_params_config['output_params']['psf_models_dir']]
    # TODO: ispect BUG in PSFEx, it replaces PSF_SUFFIX only last extension part (after last .)
    step_params_config['software_params']['config']['PSF_SUFFIX'] = ext_split(output_psf_models_format)[-1]
    step_params_config['software_params']['config']['PSF_DIR'] = output_psf_models_dir
    opmfs = build_filenames_list(input_images_names, new_ext=output_psf_models_format, 
                                    new_path=output_psf_models_dir)
    
    # output is list of calculated psf models full pathnames
    #
    
    calculate_psf_separately = False
    if 'calculate_psf_separately' in step_params_config['output_params'].keys():
        if step_params_config['output_params']['calculate_psf_separately'] == 'true':
            calculate_psf_separately = True
            
    software_params = step_params_config['software_params'] # TODO: deepcopy or not?
    
    if calculate_psf_separately:
        for icf in icfs:
            psfex = aw.api.Astromatic(**software_params)
            #this_cmd, kwargs2 = psfex.build_cmd(icf, **software_params)
            #print this_cmd
            psfex.run(icf)
    else:
        psfex = aw.api.Astromatic(**software_params)
        #this_cmd, kwargs2 = psfex.build_cmd(' '.join(icfs), **software_params)
        #print this_cmd
        psfex.run(' '.join(icfs)) # TODO: create file with catalogs list, and pass as @catalogs_list
    
    return opmfs

<h3>Test of pipeline</h3>

Input images are saved to temp folder, beause futher processing needs images and .head files in same directory.<br>
Define list of input images, raw and tech frames

In [19]:
input_images_names = sorted(get_fits_images_from_dir(paths['temp'], pipeline_config['input_images_format'], True, True))
input_images_names = filter_fits_by_header(input_images_names, paths['temp'], EXPTYPE='Light')
input_images_names = [change_file_extension(os.path.basename(iin), '') for iin in input_images_names]
input_images_names

['GRB130427_R60_001_001',
 'GRB130427_R60_001_002',
 'GRB130427_R60_001_003',
 'GRB130427_R60_001_004',
 'GRB130427_R60_001_005',
 'GRB130427_R60_001_006',
 'GRB130427_R60_001_007',
 'GRB130427_R60_001_008',
 'GRB130427_R60_001_009',
 'GRB130427_R60_001_010',
 'GRB130427_R60_001_011',
 'GRB130427_R60_001_012',
 'GRB130427_R60_001_013',
 'GRB130427_R60_001_014',
 'GRB130427_R60_001_015',
 'GRB130427_R60_001_016']

Pipeline basic functions calls

In [138]:
# stage: images_calibration
catalogs_1 = create_catalogs(input_images_names, paths, pipeline_config, 'images_calibration', 'create_calibration_catalogs')
calculate_calibration(input_images_names, paths, pipeline_config, 'images_calibration', 'create_calibration_headers')
swarped_images_1, _ = swarp_images(input_images_names, paths, pipeline_config, 'images_calibration', 'calibrate_images')
catalogs_2 = create_catalogs(input_images_names, paths, pipeline_config, 'images_calibration', 'create_calibrated_images_catalogs')
calculate_psf(input_images_names, paths, pipeline_config, 'images_calibration', 'model_psf_calibrated_images_catalogs')
catalogs_3 = create_catalogs(input_images_names, paths, pipeline_config, 'images_calibration', 'create_calibrated_images_catalogs_psf')
# stage: images_coaddition
swarped_images_2, _ = swarp_images(input_images_names, paths, pipeline_config, 'images_coaddition', 'coadd_calibrated_images')
catalogs_4 = create_catalogs(swarped_images_2, paths, pipeline_config, 'images_coaddition', 'create_coadded_image_catalog')
calculate_psf(swarped_images_2, paths, pipeline_config, 'images_coaddition', 'model_psf_coadded_image_catalog')
catalogs_5 = create_catalogs(swarped_images_2, paths, pipeline_config, 'images_coaddition', 'create_coadded_image_catalog_psf')

In [154]:
output_dict = {}
cal_images = {im: fits_ImageHDU_from_file(im) for im in swarped_images_1}
cal_cats = {cat: aw.utils.ldac.get_table_from_ldac(cat) for cat in catalogs_2}
cal_cats_psf = {cat: aw.utils.ldac.get_table_from_ldac(cat) for cat in catalogs_3}
output_dict['calibrated_images'] = cal_images
output_dict['calibrated_images_catalogs'] = cal_cats
output_dict['calibrated_images_catalogs_psf'] = cal_cats_psf

In [155]:
def fits_ImageHDU_from_file(filename, mode='denywrite', lazy_load_hdus=False):
    f = fits.open(filename, mode='denywrite', lazy_load_hdus=False)
    fits_image = fits.ImageHDU(data=f[0].data, header=f[0].header)
    f.close()
    return fits_image

In [156]:
output_dict_2 = {}
cal_images = {im: fits_ImageHDU_from_file(im) for im in swarped_images_2}
cal_cats = {cat: aw.utils.ldac.get_table_from_ldac(cat) for cat in catalogs_4}
cal_cats_psf = {cat: aw.utils.ldac.get_table_from_ldac(cat) for cat in catalogs_5}
output_dict_2['calibrated_images'] = cal_images
output_dict_2['calibrated_images_catalogs'] = cal_cats
output_dict_2['calibrated_images_catalogs_psf'] = cal_cats_psf

In [120]:
with open(catalogs_4[0], 'rb') as fid:
    data = BytesIO(fid.read())

In [103]:
for key in res_dict.keys():
    print key, len(res_dict[key].keys())
for key in output_dict_4.keys():
    print key, len(output_dict_4[key].keys())

calibrated_images_catalogs_psf 17
calibrated_images 17
calibrated_images_catalogs 17
calibrated_images_catalogs_psf 16
calibrated_images 16
calibrated_images_catalogs 16


In [157]:
output_dict_3 = copy.deepcopy(output_dict_2)
output_dict_4 = copy.deepcopy(output_dict)

In [158]:
# each input dict is a dict{type:dict{fn:bytes}}
def ggg(dict1, dict2):
    res = {}
    for k, v in dict1.iteritems():
        if k not in res.keys():
            res[k] = {}
        for kk, vv in dict1[k].iteritems():
            res[k][kk] = vv

    for k, v in dict2.iteritems():
        if k not in res.keys():
            res[k] = {}
        for kk, vv in dict2[k].iteritems():
            res[k][kk] = vv
    return res

In [159]:
res_dict = ggg(output_dict_3, output_dict_4)

In [163]:
f = res_dict['calibrated_images'].items()
image = f[0][1]

In [167]:
fits.writeto('test.fits', data=image.data, header=image.header)

In [179]:
tbl_tuple = res_dict['calibrated_images_catalogs_psf'].items()[0]
aw.utils.ldac.save_table_as_ldac(tbl_tuple[1], 'test_cat.cat')

Check coadded image catalog

In [180]:
catalog = aw.utils.ldac.get_table_from_ldac('test_cat.cat')
catalog

NUMBER,EXT_NUMBER,XWIN_IMAGE,YWIN_IMAGE,ERRAWIN_IMAGE,ERRBWIN_IMAGE,ERRTHETAWIN_IMAGE,XWIN_WORLD,YWIN_WORLD,FLUX_AUTO,FLUXERR_AUTO,FLAGS,FLAGS_WEIGHT,FLUX_RADIUS,ELONGATION,MAG_AUTO,MAGERR_AUTO,ALPHAPSF_SKY,DELTAPSF_SKY,ERRX2PSF_WORLD,ERRY2PSF_WORLD,FLUX_PSF,FLUXERR_PSF,MAG_PSF,MAGERR_PSF,FLUXERR_APER,FLUX_APER,"VIGNET [20,20]",SNR_WIN
,,pix,pix,pix,pix,deg,deg,deg,ct,ct,,,pix,,mag,mag,deg,deg,deg2,deg2,ct,ct,mag,mag,ct,ct,ct,
int32,int16,float64,float64,float32,float32,float32,float64,float64,float32,float32,int16,int16,float32,float32,float32,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32
1,1,199.010602485,37.9179525588,0.0102173,0.0101383,-72.7561,173.159134955,27.6451661357,1796.45,13.5765,4,0,2.69306,1.51522,-8.13603,0.00820735,173.159179092,27.6453762634,0.0,0.0,0.0,0.0,99.0,99.0,3.24317,830.273,0.611328 .. 0.419922,299.755
2,1,384.74375879,11.4640961905,0.0801357,0.0793144,-59.5147,173.103225995,27.6381057041,109.568,5.70855,4,0,1.93767,1.17387,-5.09921,0.0565809,173.103194583,27.6381976196,0.0,0.0,0.0,0.0,99.0,99.0,2.77788,66.6823,-1.14258 .. 0.621094,29.7116
3,1,166.723685832,12.7728482889,0.0465225,0.0454836,44.4767,173.16885299,27.6384595617,85.0137,4.074,4,0,1.27141,1.61323,-4.82372,0.052043,173.169006949,27.638351102,0.0,0.0,0.0,0.0,99.0,99.0,2.76513,56.0308,-16.8457 .. 0.699219,34.5657
4,1,279.968759826,11.9314666831,0.0777,0.0776669,50.6996,173.134764638,27.6382365261,39.8438,2.49137,4,0,0.757805,1.01051,-4.0009,0.0679059,173.134907573,27.6381190897,0.0,0.0,0.0,0.0,99.0,99.0,2.74466,38.245,-32.5586 .. 0.960938,38.5539
5,1,224.015763844,497.728014515,0.191113,0.190813,-17.0798,173.151613486,27.7677824754,195.299,9.30909,4,0,3.92954,1.50413,-5.72675,0.0517651,173.151550933,27.7678003745,0.0,0.0,0.0,0.0,99.0,99.0,2.76419,46.652,-1.09375 .. -0.162109,24.8686
6,1,265.106121939,494.994342509,0.0258518,0.0251956,-89.6495,173.13923008,27.7670534141,44.8711,2.2562,4,0,0.750279,1.64554,-4.12992,0.0546062,173.139139444,27.7671156981,0.0,0.0,0.0,0.0,99.0,99.0,2.76045,45.1969,-0.142578 .. 1.99023,36.2554
7,1,413.670294402,492.016041988,0.166282,0.165614,72.532,173.094457798,27.7662496947,93.8516,5.53493,4,0,2.93524,1.36369,-4.9311,0.0640473,173.094394051,27.7663293096,0.0,0.0,0.0,0.0,99.0,99.0,2.75319,46.6174,-0.5 .. 0.0273438,21.3961
8,1,132.402703786,491.673258806,0.0961593,0.095979,-65.5327,173.17922254,27.766164091,107.361,6.48395,4,0,2.1245,1.27958,-5.07712,0.0655876,173.179357798,27.7661493979,0.0,0.0,0.0,0.0,99.0,99.0,2.77549,67.1825,0.078125 .. 0.160156,27.0353


### Local hardcoded pipelines (testing astromatic software)

In [68]:
# by defualt, we search for images in paths['images']
def pipeline_1(paths):
    print "prepare..."
    # prepare
    fits_format = 'fit'
    filenames = sorted(get_fits_images_from_dir(paths['images'], files_format=fits_format))
    exposures = []
    for filename in filenames:
        exposures.append({'image': os.path.join(paths['images'], filename)})
    # STEP 1: basic extraction with SExtractor (generate catalogs) for each image
    catalog_names = []
    print 'begin SExtractor execution'
    for files in exposures:
        print files['image'] + ':'
        # Create names for the output catalogs for each image (in temp dir)
        catalog_names.append(os.path.join(paths['temp'],
            os.path.basename(files['image']).replace('.'+fits_format, '.cat')))
        # set params for sextractor execution
        sex_kwargs_1 = {'code': 'SExtractor'}
        sex_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.sex')
        sex_kwargs_1['config'] = {'CATALOG_NAME': catalog_names[-1]}
        sex_kwargs_1['config']['CATALOG_TYPE'] = 'FITS_LDAC'
        sex_kwargs_1['config']['FILTER'] = 'N'
        sex_kwargs_1['temp_path'] = paths['temp']
        sex_kwargs_1['params'] = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                                  'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'ERRA_WORLD', 'ERRB_WORLD', 
                                  'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                                  'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                                  'FLAGS', 'FLUX_RADIUS', 'ELONGATION']
        sextractor = aw.api.Astromatic(**sex_kwargs_1)
        print sextractor.run(files['image'])
    print 'all catalogs successfully created'
    
    # STEP 2: compute solution for extracted catalogs
    print 'SCAMP now working...'
    scamp_kwargs_1 = {
        'config': {
            'ASTREF_CATALOG': 'USNO-B1',
            'ASTREF_BAND': 'DEFAULT',
            'SOLVE_PHOTOM': 'N',
            'CHECKPLOT_DEV': 'NULL'
            }
    }
    scamp_kwargs_1['code'] = 'SCAMP'
    scamp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.scamp')
    scamp = aw.api.Astromatic(**scamp_kwargs_1)
    this_cmd, kwargs2 = scamp.build_cmd(catalog_names, **scamp_kwargs_1)
    #print this_cmd
    print scamp.run(catalog_names)
    
    print 'SWarp now working...'
    # STEP 3: use SWarp for coadd images using .head files, computed with SCAMP
    coadd_fits_name = 'coadded.fits'
    coadd_fits_filename = os.path.join(paths['temp'], coadd_fits_name)
    swarp_kwargs_1 = {
        'config': {
            'IMAGEOUT_NAME': coadd_fits_filename,
            'WEIGHTOUT_NAME': coadd_fits_filename.replace('.fits','.wtmap.fits'),
        }
        }
    swarp_kwargs_1['code'] = 'SWarp'
    swarp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.swarp')
    swarp = aw.api.Astromatic(**swarp_kwargs_1)
    this_cmd, kwargs2 = swarp.build_cmd([exp['image'] for exp in exposures], **swarp_kwargs_1)
    #print this_cmd
    print swarp.run([exp['image'] for exp in exposures])
    #return
    # STEP 4: extract object from coadded image using PSF
    print 'SExtractor now working...'
    coadd_weight_map = coadd_fits_filename.replace('.fits','.wtmap.fits')
    sex_kwargs_2 = {
            'config': {
                'CATALOG_NAME': coadd_fits_filename.replace('.fits', '.cat'),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'WEIGHT_IMAGE': coadd_weight_map,
                'WEIGHT_TYPE': 'MAP_WEIGHT',        
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_APER(1)',
                'FLUXERR_APER(1)', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'VIGNET(20,20)', 'SNR_WIN']
        }
    sex_kwargs_2['code'] = 'SExtractor'
    sex_kwargs_2['config_file'] = os.path.join(paths['config'], 'default.sex')
    sex_kwargs_2['temp_path'] = paths['temp']
    sextractor = aw.api.Astromatic(**sex_kwargs_2)
    print sextractor.run(coadd_fits_filename)
    
    # Calculate PSF
    print 'PSFex now working...'
    psf_kwargs_1 = {
        'config': {
            'PHOT_APERTURES':'15'
            'CENTER_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
            'PSFVAR_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
            'PHOTFLUX_KEY': 'FLUX_AUTO',
            'PHOTFLUXERR_KEY': 'FLUXERR_AUTO',
            'CHECKPLOT_DEV': 'NULL',
            'STABILITY_TYPE':  'EXPOSURE',
            'PSF_SUFFIX': '.psf'
        }
    }
    psf_kwargs_1['code'] = 'PSFEx'
    psf_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.psfex')
    psf_kwargs_1['temp_path'] = paths['temp']
    psfex = aw.api.Astromatic(**psf_kwargs_1)
    print psfex.run(coadd_fits_filename.replace('.fits', '.cat'))
    
    # Calculate PSF photometry for coadded image
    print 'SExtractor now working...'
    output_cat_name = 'result.ldac.fits'
    catalog_name = os.path.join(paths['catalogs'], output_cat_name)
    sex_kwargs_3 = {
            'config': {
                'PSF_NAME': os.path.join(coadd_fits_filename.replace('.fits', '.psf')),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'CATALOG_NAME': catalog_name,
                'WEIGHT_IMAGE': coadd_weight_map,
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                'FLUXERR_AUTO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'MAG_AUTO', 'MAGERR_AUTO', 'ALPHAPSF_SKY', 'DELTAPSF_SKY',
                'ERRX2PSF_WORLD','ERRY2PSF_WORLD', 'FLUX_PSF', 'FLUXERR_PSF', 'MAG_PSF', 'MAGERR_PSF']
        }
    sex_kwargs_3['code'] = 'SExtractor'
    sex_kwargs_3['config_file'] = os.path.join(paths['config'], 'default.sex')
    sex_kwargs_3['temp_path'] = paths['temp']
    sextractor = aw.api.Astromatic(**sex_kwargs_3)
    print sextractor.run(coadd_fits_filename)
    return catalog_name

In [70]:
catalog_name = pipeline_1(paths)
#catalog = aw.utils.ldac.get_table_from_ldac(catalog_name)
#catalog

prepare...
begin SExtractor execution
/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_001.fit:
{'status': 'success'}
/home/ser/Dev/astro_engine/spark_pipeline/spark_pipeline_test_1/images/GRB130427_R60_001_002.fit:
{'status': 'success'}
all catalogs successfully created
SCAMP now working...
{'status': 'success'}
SWarp now working...
{'status': 'success'}


## Test pipeline components

In [29]:
def build_pipeline(pipeline, exposures, ref_catalog='IKI', ref_band='DEFAULT',
        stack_name = 'test_stack.fits', output_cat_name='test_psf.ldac.fits'):
    # Generate catalogs from sextractor
    catalog_names = []
    for files in exposures:
        # Create names for the output catalogs for each image
        catalog_names.append(os.path.join(pipeline.paths['temp'],
            os.path.basename(files['image']).replace('.fit', '.cat')))
        kwargs = {
            # image to SExtract
            'files': files,
            # Arguments to initialize Astromatic class
            'api_kwargs': {
                # Configuration parameters
                'config': {
                    'CATALOG_NAME': catalog_names[-1],
                    'CATALOG_TYPE': 'FITS_LDAC',
                    'FILTER': 'N',
                    #'WEIGHT_TYPE': 'MAP_WEIGHT',
                },
                # Output parameters
                'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                    'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                    'FLUXERR_AUTO', 'IMAFLAGS_ISO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                    'ELONGATION'],
            },
        }
        # Add the step to the pipeline
        pipeline.add_step(aw.api.run_sex, ['step1', 'SExtractor'], **kwargs)

    # Use SCAMP to get astrometric solutions
    kwargs = {
        'catalogs': catalog_names,
        'api_kwargs': {
            'config': {
                'ASTREF_CATALOG': ref_catalog,
                'ASTREF_BAND': ref_band,
                'SOLVE_PHOTOM': 'N',
                'CHECKPLOT_DEV': 'NULL'
            },
        }
    }
    pipeline.add_step(aw.api.run_scamp, ['step2', 'SCAMP'],**kwargs)

    # Resample (rotate and scale) and combine (stack) images using SWarp
    stack_filename = os.path.join(pipeline.paths['temp'], stack_name)
    kwargs = {
        'filenames': [exp['image'] for exp in exposures],
        'api_kwargs': {
            'config': {
                'WEIGHT_TYPE': 'MAP_WEIGHT',
                'WEIGHT_SUFFIX': '.wtmap.fits',
                'IMAGEOUT_NAME': stack_filename,
                'WEIGHTOUT_NAME': stack_filename.replace('.fits','.wtmap.fits'),
            },
        }
    }
    pipeline.add_step(aw.api.run_swarp, ['step3', 'SWarp'], **kwargs)

    # Get positions in stack for PSF photometry (SExtractor -> PSFex -> SExtractor)
    kwargs = {
        'files': {
            'image': stack_filename,
            'wtmap': stack_filename.replace('.fits', '.wtmap.fits')
        },
        'api_kwargs': {
            'config': {
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_APER(1)',
                'FLUXERR_APER(1)', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'VIGNET(20,20)', 'SNR_WIN'],
        }
    }
    pipeline.add_step(aw.api.run_sex, ['step4', 'SExtractor'], **kwargs)

    # Calculate PSF
    kwargs = {
        'catalogs': stack_filename.replace('.fits', '.cat'),
        'api_kwargs': {
            'config': {
                'CENTER_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
                'PSFVAR_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
                'CHECKPLOT_DEV': 'NULL',
                'PSF_SUFFIX': '.psf'
            }
        }
    }
    pipeline.add_step(aw.api.run_psfex, ['step5', 'PSFEx'], **kwargs)

    # Calculate PSF photometry for coadded image
    catalog_name = os.path.join(pipeline.paths['catalogs'], output_cat_name)
    kwargs = {
        'files': {
            'image': stack_filename,
            'wtmap': stack_filename.replace('.fits', '.wtmap.fits')
        },
        'api_kwargs': {
            'config': {
                'PSF_NAME': os.path.join(stack_filename.replace('.fits', '.psf')),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'CATALOG_NAME': catalog_name,
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                'FLUXERR_AUTO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'MAG_AUTO', 'MAGERR_AUTO', 'ALPHAPSF_SKY', 'DELTAPSF_SKY',
                'ERRX2PSF_WORLD','ERRY2PSF_WORLD', 'FLUX_PSF', 'FLUXERR_PSF', 'MAG_PSF', 'MAGERR_PSF'],
        }
    }
    pipeline.add_step(aw.api.run_sex, ['step6', 'SExtractor'], **kwargs)

    def save_output(pipeline, old_stack, new_stack, old_cat, new_cat):
        # Copy the final stack and catalog from the temp folder
        import shutil
        # Move the weight map if it exists
        if os.path.isfile(old_stack.replace('.fits', '.wtmap.fits')):
            shutil.move(old_stack.replace('.fits', '.wtmap.fits'), new_stack.replace('.fits', '.wtmap.fits'))
        shutil.move(old_stack, new_stack)
        shutil.move(old_cat, new_cat)
        result = {
            'status': 'success'
        }
        return result

    kwargs = {
        'old_stack': stack_filename,
        'new_stack': os.path.join(pipeline.paths['stacks'], stack_name),
        'old_cat': catalog_name,
        'new_cat': os.path.join(pipeline.paths['catalogs'], output_cat_name)
    }
    pipeline.add_step(save_output, ['step7', 'save_output'], **kwargs)

    return pipeline